In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timezone, timedelta
import dateutil.parser
import time


links = ['https://tsn.ua/news', 'https://tsn.ua/news/page-2']
article_hrefs = []
for link in links:
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")
    feed_list = soup.find("ul", class_='l-entries l-entries--bordered')
    articles = feed_list.find_all("li", class_='l-entries__item')
    for article in articles:
        title_tag = article.find('h2')
        if title_tag is not None:
            href = title_tag.find('a')['href']
            timestamp = article.find('time', class_='text-current c-bar__link c-entry__time')['datetime']
            article_hrefs.append((href, timestamp))

In [2]:
print(article_hrefs)

[('https://tsn.ua/svit/u-chechni-17-richnyy-mesnyk-napav-na-sylovykiv-kadyrova-khloptsia-vbyly-a-rodychiv-zorstoko-pokaraly-2805155.html', '2025-04-10T15:11:31+03:00'), ('https://tsn.ua/ukrayina/chy-mozna-na-velykden-varyty-kholodets-z-pivnia-svshchennyy-vidpoviv-2805620.html', '2025-04-10T15:02:08+03:00'), ('https://tsn.ua/glamur/druzyna-zirky-lihy-smikhu-chechuna-nazvala-prychynu-yoho-smerti-mayze-rik-borolysia-za-zyttia-2805611.html', '2025-04-10T15:00:26+03:00'), ('https://tsn.ua/auto/news/iak-odniyeiu-frazoiu-zupynyty-avtoshakhrayiv-porada-vid-politsiyi-2805608.html', '2025-04-10T15:00:05+03:00'), ('https://tsn.ua/lady/zdorovye/zdorovyi-obraz-zhizni/vsyoho-dvi-nevelyki-zminy-v-sposobi-zyttia-iak-pidtrymaty-stabilnyy-riven-tsukru-v-krovi-2805563.html', '2025-04-10T15:00:00+03:00'), ('https://tsn.ua/other/iak-posiiaty-morkvu-dlia-shvydkoho-skhodu-porady-ta-layfkhaky-vid-ekspertiv-2805002.html', '2025-04-10T15:00:00+03:00'), ('https://tsn.ua/lady/news/obschestvo/ameliia-v-shtannomu-k

In [5]:
article_page = requests.get(article_hrefs[0][0])
soup = BeautifulSoup(article_page.content, "html.parser")
title_tag = soup.find('h1', class_='c-entry__title c-title c-title--h1 font-bold')
title = title_tag.get_text()
paragraphs_list = []
main_content = soup.find('div', class_='c-prose c-post__inner')
lead_div = soup.find_all('div', class_='c-entry__lead c-prose__lead')
paragraphs_list.append(lead_div[0].find('p').get_text())
paragraphs = main_content.find_all('p', recursive=False)
sources = []
for paragraph in paragraphs:
    text = paragraph.get_text()
    text = text.replace(' ', ' ').strip()
    if len(text) != 0 and not text.startswith('Читайте також'):
        paragraphs_list.append(text)
    links = paragraph.find_all('a')
    for link in links:
        source_url = link['href']
        if source_url:
            sources.append(source_url)

In [17]:
article_time = dateutil.parser.isoparse(timestamp)
current_time = datetime.now(timezone.utc)
one_hour_ago = current_time - timedelta(hours=3)
if article_time >= one_hour_ago:
    print({
        'href': href,
        'timestamp': timestamp,
        'title': title,
        'paragraphs': paragraphs_list,
        'sources': sources
    })

{'href': 'https://tsn.ua/zdorovya/chomu-ne-vdayetsia-pozbutysia-zyru-na-zyvoti-poiasnennia-endokrynolohyni-2805572.html', 'timestamp': '2025-04-10T14:22:30+03:00', 'title': '\n        У Чечні 17-річний месник напав на силовиків Кадирова: хлопця вбили, а родичів жорстоко покарали\n            ', 'paragraphs': ['Кадировці зігнали все село дивитися на\xa0тіло юного месника.', 'Глава Чечні Рамзан Кадиров продовжує тримати у страху населення Чечні, проте про факти спротиву досі виникають, попри придушення чеченського визвольного руху проти РФ. Кадиров, як вірна маріонетка Кремля, бореться з цими проявами жорсткими середньовічними методами.', 'Так, 8 квітня стало відомо, що 17-річний чеченський юнак напав із ножем на кадирівських силовиків. Один із поліцейських помер від поранень. Юнака було вбито вогнем у відповідь.', '«Дон-дон» одразу ж заявив, що за діями нападника нібито керував громадянин України.', 'Кадиров, навіть всупереч російським законам, доручив вигнати з Чечні родичів загиблого 

In [24]:
def scrape_tsn(scraping_delay=0.25):
    links = ['https://tsn.ua/news', 'https://tsn.ua/news/page-2']
    print(scraping_delay)
    article_hrefs = []
    for link in links:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, "html.parser")
        feed_list = soup.find("ul", class_='l-entries l-entries--bordered')
        articles = feed_list.find_all("li", class_='l-entries__item')
        for article in articles:
            title_tag = article.find('h2')
            if title_tag is not None:
                href = title_tag.find('a')['href']
                timestamp = article.find('time', class_='text-current c-bar__link c-entry__time')['datetime']
                article_hrefs.append((href, timestamp))
    article_data = []
    print(article_hrefs)
    for (href, timestamp) in article_hrefs:
        article_page = requests.get(href)
        soup = BeautifulSoup(article_page.content, "html.parser")
        title_tag = soup.find('h1', class_='c-entry__title c-title c-title--h1 font-bold')
        title = title_tag.get_text()
        paragraphs_list = []
        main_content = soup.find('div', class_='c-prose c-post__inner')
        lead_div = soup.find_all('div', class_='c-entry__lead c-prose__lead')
        paragraphs_list.append(lead_div[0].find('p').get_text())
        paragraphs = main_content.find_all('p', recursive=False)
        sources = []
        for paragraph in paragraphs:
            text = paragraph.get_text()
            text = text.replace(' ', ' ').strip()
            if len(text) != 0 and not text.startswith('Читайте також'):
                paragraphs_list.append(text)
            links = paragraph.find_all('a')
            for link in links:
                source_url = link['href']
                if source_url:
                    sources.append(source_url)
        article_time = dateutil.parser.isoparse(timestamp)
        current_time = datetime.now(timezone.utc)
        one_hour_ago = current_time - timedelta(hours=scraping_delay)
        if article_time >= one_hour_ago:
            article_data.append({
                'href': href,
                'timestamp': timestamp,
                'title': title,
                'paragraphs': paragraphs_list,
                'sources': sources
            })
        else:
            return article_data
    return article_data

In [32]:
article_time = dateutil.parser.isoparse('2025-04-10T15:21:06+03:00')
current_time = datetime.now(timezone.utc)
one_hour_ago = current_time - timedelta(hours=0)
if article_time >= one_hour_ago:
    print('eee')

In [25]:
start = time.time()
print(scrape_tsn(scraping_delay=4))
end = time.time()
print(end - start)

4
[('https://tsn.ua/nauka_it/naukovtsi-rozkryly-zahadku-volokhatykh-knyh-chasiv-serednyovichchia-foto-2805632.html', '2025-04-10T15:21:06+03:00'), ('https://tsn.ua/lady/dom_i_deti/dom/drakoniachi-iaytsia-tsikavyy-metod-farbuvannia-do-velykodnia-2803883.html', '2025-04-10T15:19:04+03:00'), ('https://tsn.ua/lady/zvezdy/zvezdy/poza-chasom-hto-iz-zirok-nosit-kultovu-prikrasu-cvyah-vid-cartier-1950976.html', '2025-04-10T15:16:00+03:00'), ('https://tsn.ua/svit/u-chechni-17-richnyy-mesnyk-napav-na-sylovykiv-kadyrova-khloptsia-vbyly-a-rodychiv-zorstoko-pokaraly-2805155.html', '2025-04-10T15:11:31+03:00'), ('https://tsn.ua/ukrayina/chy-mozna-na-velykden-varyty-kholodets-z-pivnia-svshchennyy-vidpoviv-2805620.html', '2025-04-10T15:02:08+03:00'), ('https://tsn.ua/glamur/druzyna-zirky-lihy-smikhu-chechuna-nazvala-prychynu-yoho-smerti-mayze-rik-borolysia-za-zyttia-2805611.html', '2025-04-10T15:00:26+03:00'), ('https://tsn.ua/auto/news/iak-odniyeiu-frazoiu-zupynyty-avtoshakhrayiv-porada-vid-politsiyi